# Florida's 27th District Election Results Data Cleaning

This notebook is dedicated to cleaning and preparing the election results data from Florida's 27th district. The dataset contains information on candidates running for both the **House of Representatives (USR)** and the **Presidency (PRE)**. Our goal is to extract the relevant information that will be used for **predicting future election outcomes** in both races.

### Data Overview:
- **Juris1num**: The district number where the election is held.
- **CanNameLast, CanNameFirst, CanNameMiddle**: These columns represent the full name of the candidate.
- **CanVotes**: Total votes received by each candidate.
- **PartyCode and PartyName**: The code and full name of the political party the candidate represents.

By cleaning and organizing this data, we aim to create a robust foundation for predictive models that will help forecast election results based on historical voting patterns, party affiliations, and candidate details.


In [ ]:
import pandas as pd
import re
import os

In [ ]:

df = pd.read_csv('11022010Election.txt', sep='\t', engine='python', comment='#', on_bad_lines='warn')
df

In [ ]:
# Select the desired columns and create a new DataFrame
selected_columns = ['ElectionDate','PartyCode', 'PartyName', 'RaceCode', 'CountyCode', 'CountyName', 'Juris1num', 'CanVotes', 'CanNameLast', 'CanNameFirst']
new_df = df[selected_columns].copy()

new_df

,ElectionDate,PartyCode,PartyName,RaceCode,CountyCode,CountyName,Juris1num,CanVotes,CanNameLast,CanNameFirst
0,11/2/2010,REP,Republican,USS,ALA,Alachua,NaN,29825,Rubio,Marco
1,11/2/2010,REP,Republican,USS,BAK,Baker,NaN,5435,Rubio,Marco
2,11/2/2010,REP,Republican,USS,BAY,Bay,NaN,40408,Rubio,Marco
3,11/2/2010,REP,Republican,USS,BRA,Bradford,NaN,5156,Rubio,Marco
4,11/2/2010,REP,Republican,USS,BRE,Brevard,NaN,107930,Rubio,Marco
...,...,...,...,...,...,...,...,...,...,...
5704,11/2/2010,NOP,Non-Partisan,A01,WAS,Washington,NaN,3370,No for Rejection,NaN
5705,11/2/2010,NOP,Non-Partisan,A06,WAS,Washington,NaN,3987,No for Rejection,NaN
5706,11/2/2010,NOP,Non-Partisan,A05,WAS,Washington,NaN,4199,No for Rejection,NaN
5707,11/2/2010,NOP,Non-Partisan,A08,WAS,Washington,NaN,3255,No for Rejection,NaN


In [ ]:
# prompt: The election date has format: DD/MM/YYYY and i only want to keeo the year

# Assuming your DataFrame is named 'new_df' and the column with election date is 'ElectionDate'
new_df['ElectionYear'] = pd.to_datetime(new_df['ElectionDate']).dt.year
new_df

,ElectionDate,PartyCode,PartyName,RaceCode,CountyCode,CountyName,Juris1num,CanVotes,CanNameLast,CanNameFirst,ElectionYear
0,11/2/2010,REP,Republican,USS,ALA,Alachua,NaN,29825,Rubio,Marco,2010
1,11/2/2010,REP,Republican,USS,BAK,Baker,NaN,5435,Rubio,Marco,2010
2,11/2/2010,REP,Republican,USS,BAY,Bay,NaN,40408,Rubio,Marco,2010
3,11/2/2010,REP,Republican,USS,BRA,Bradford,NaN,5156,Rubio,Marco,2010
4,11/2/2010,REP,Republican,USS,BRE,Brevard,NaN,107930,Rubio,Marco,2010
...,...,...,...,...,...,...,...,...,...,...,...
5704,11/2/2010,NOP,Non-Partisan,A01,WAS,Washington,NaN,3370,No for Rejection,NaN,2010
5705,11/2/2010,NOP,Non-Partisan,A06,WAS,Washington,NaN,3987,No for Rejection,NaN,2010
5706,11/2/2010,NOP,Non-Partisan,A05,WAS,Washington,NaN,4199,No for Rejection,NaN,2010
5707,11/2/2010,NOP,Non-Partisan,A08,WAS,Washington,NaN,3255,No for Rejection,NaN,2010


In [ ]:
# Filter the DataFrame to include only rows with 'PRE' or 'USR' in the 'RaceCode' column
filtered_df = new_df[new_df['RaceCode'].isin(['PRE', 'USR'])]

filtered_df

,ElectionDate,PartyCode,PartyName,RaceCode,CountyCode,CountyName,Juris1num,CanVotes,CanNameLast,CanNameFirst,ElectionYear
1139,11/2/2010,REP,Republican,USR,ESC,Escambia,1.0,68628,Miller,Jeff,2010
1140,11/2/2010,REP,Republican,USR,HOL,Holmes,1.0,5072,Miller,Jeff,2010
1141,11/2/2010,REP,Republican,USR,OKA,Okaloosa,1.0,42501,Miller,Jeff,2010
1142,11/2/2010,REP,Republican,USR,SAN,Santa Rosa,1.0,40188,Miller,Jeff,2010
1143,11/2/2010,REP,Republican,USR,WAL,Walton,1.0,8360,Miller,Jeff,2010
...,...,...,...,...,...,...,...,...,...,...,...
1501,11/2/2010,FWP,Florida Whig Party,USR,DAD,Miami-Dade,25.0,2696,Porter,Craig,2010
1502,11/2/2010,FWP,Florida Whig Party,USR,MON,Monroe,25.0,2,Porter,Craig,2010
1503,11/2/2010,TEA,Tea Party,USR,CLL,Collier,25.0,961,Arrojo,Roly,2010
1504,11/2/2010,TEA,Tea Party,USR,DAD,Miami-Dade,25.0,3351,Arrojo,Roly,2010


In [ ]:
miami_dade_df = filtered_df[filtered_df['CountyName'] == 'Miami-Dade']
miami_dade_df

,ElectionDate,PartyCode,PartyName,RaceCode,CountyCode,CountyName,Juris1num,CanVotes,CanNameLast,CanNameFirst,ElectionYear
1445,11/2/2010,DEM,Democrat,USR,DAD,Miami-Dade,17.0,82682,Wilson,Frederica,2010
1447,11/2/2010,NPA,No Party Affiliation,USR,DAD,Miami-Dade,17.0,10961,Vereen,Roderick,2010
1448,11/2/2010,REP,Republican,USR,DAD,Miami-Dade,18.0,85051,Ros-Lehtinen,Ileana,2010
1450,11/2/2010,DEM,Democrat,USR,DAD,Miami-Dade,18.0,38756,Banciella,Rolando,2010
1459,11/2/2010,REP,Republican,USR,DAD,Miami-Dade,20.0,8906,Harrington,Karen,2010
1461,11/2/2010,DEM,Democrat,USR,DAD,Miami-Dade,20.0,15890,Wasserman Schultz,NaN,2010
1463,11/2/2010,NPA,No Party Affiliation,USR,DAD,Miami-Dade,20.0,387,Blumenthal,Stanley,2010
1465,11/2/2010,NPA,No Party Affiliation,USR,DAD,Miami-Dade,20.0,277,Kunst,Robert,2010
1467,11/2/2010,WRI,Write-In,USR,DAD,Miami-Dade,20.0,0,Schock,Clayton,2010
1495,11/2/2010,REP,Republican,USR,DAD,Miami-Dade,25.0,63364,Rivera,David,2010


In [ ]:
import pandas as pd
import os
def clean_election_data(df):

  # Select the desired columns and create a new DataFrame
  selected_columns = ['ElectionDate','PartyCode', 'PartyName', 'RaceCode', 'CountyCode', 'CountyName', 'Juris1num', 'CanVotes', 'CanNameLast', 'CanNameFirst']
  new_df = df[selected_columns].copy()

  # Extract the election year from the 'ElectionDate' column
  new_df['ElectionYear'] = pd.to_datetime(new_df['ElectionDate']).dt.year

  # Filter the DataFrame to include only rows with 'PRE' or 'USR' in the 'RaceCode' column
  filtered_df = new_df[new_df['RaceCode'].isin(['PRE', 'USR'])]

  # Filter further to include only data from Miami-Dade county
  miami_dade_df = filtered_df[filtered_df['CountyName'] == 'Miami-Dade']

  return miami_dade_df

final_df = pd.DataFrame()

for filename in os.listdir('.'):
  if filename.endswith('.txt'):
    try:
      df = pd.read_csv(filename, sep='\t', engine='python', comment='#', on_bad_lines='warn', encoding='latin-1')
      cleaned_df = clean_election_data(df)
      print(f"Cleaned data for {filename}:")
      print(cleaned_df)
      final_df = pd.concat([final_df, cleaned_df], ignore_index=True)
    except FileNotFoundError:
      print(f"File '{filename}' not found.")
    except Exception as e:
      print(f"Error processing {filename}: {e}")


In [ ]:
print(final_df)

    ElectionDate PartyCode     PartyName RaceCode CountyCode  CountyName  \
0      11/8/1988       REP    Republican      PRE        DAD  Miami-Dade   
1      11/8/1988       DEM      Democrat      PRE        DAD  Miami-Dade   
2      11/8/1988       LIB   Libertarian      PRE        DAD  Miami-Dade   
3      11/8/1988       NAL  New Alliance      PRE        DAD  Miami-Dade   
4      11/8/1988       REP    Republican      USR        DAD  Miami-Dade   
..           ...       ...           ...      ...        ...         ...   
463    11/8/1994       REP    Republican      USR        DAD  Miami-Dade   
464    9/10/2002       REP    Republican      USR        DAD  Miami-Dade   
465    9/10/2002       REP    Republican      USR        DAD  Miami-Dade   
466    9/10/2002       DEM      Democrat      USR        DAD  Miami-Dade   
467    9/10/2002       DEM      Democrat      USR        DAD  Miami-Dade   

    Juris1num  CanVotes    CanNameLast        CanNameFirst  ElectionYear  
0         Na

In [7]:
final_df.to_csv('cleaned_election_data.csv', index=False)

Add voter turnout

In [ ]:
def process_excel_file(excel_file):
  try:

    # Read the Excel file
    df = pd.read_csv(excel_file,sep=';')
    # Extract the date from the filename using a regular expression
    match = re.search(r'Turnout(Primary|General)(\d{4})', excel_file)

    if match:
      election_type = match.group(1)  # Extracts "Primary" or "General"
      date = match.group(2)
      date_str = f"{election_type}{date}"




    # Select only the "County" and "Turnout%" columns
    df_selected = df[['County', 'PercentageTurnout']]

    # Get the voter turnout for Miami-Dade County
    miami_dade_turnout = df_selected[df_selected['County'] == 'Miami-Dade']['PercentageTurnout'].values

    turnout_number = float(miami_dade_turnout[0].strip('%')) / 100
    print(turnout_number)
    # If Miami-Dade County is found, return the turnout and date
    if turnout_number > 0:
      return date_str, turnout_number

    return None, None  # Return None if Miami-Dade is not found

  except Exception as e:
    print(f"Error processing {excel_file}: {e}")
    return None, None


# Get a list of all XLSX files in the current directory
xlsx_files = [f for f in os.listdir() if f.endswith('.csv')]
print(xlsx_files)
# Create an empty dictionary to store the turnout data
turnout_data = {}

# Process each XLSX file
for file in xlsx_files:
  date, turnout = process_excel_file(file)
  print(f"Date: {date}, Turnout: {turnout}")
  if date is not None and turnout is not None:
    turnout_data[date] = turnout

# Create a DataFrame from the turnout data
turnout_df = pd.DataFrame.from_dict(turnout_data, orient='index', columns=['Miami-Dade Turnout'])

turnout_df.to_csv('turnoutTraining_data.csv')

['TurnoutGeneral2020.csv', 'TurnoutGeneral2016.csv', 'TurnoutGeneral2012.csv', 'TurnoutPrimary2016.csv', 'TurnoutPrimary2022.csv', 'TurnoutGeneral2022.csv', 'TurnoutPrimary2012.csv', 'TurnoutPrimary2020.csv', 'TurnoutPrimary2014.csv', 'TurnoutGeneral2018.csv', 'TurnoutPrimary2018.csv', 'TurnoutGeneral2014.csv']
0.746
Date: General2020, Turnout: 0.746
0.7240000000000001
Date: General2016, Turnout: 0.7240000000000001
Error processing TurnoutGeneral2012.csv: index 0 is out of bounds for axis 0 with size 0
Date: None, Turnout: None
0.205
Date: Primary2016, Turnout: 0.205
0.191
Date: Primary2022, Turnout: 0.191
0.46799999999999997
Date: General2022, Turnout: 0.46799999999999997
0.2
Date: Primary2012, Turnout: 0.2
0.28300000000000003
Date: Primary2020, Turnout: 0.28300000000000003
0.14400000000000002
Date: Primary2014, Turnout: 0.14400000000000002
0.569
Date: General2018, Turnout: 0.569
0.213
Date: Primary2018, Turnout: 0.213
0.40700000000000003
Date: General2014, Turnout: 0.4070000000000000

In [ ]:
# Convert 'ElectionDate' column to datetime objects for proper merging
cleaned_df['ElectionDate'] = pd.to_datetime(cleaned_df['ElectionDate'])
turnout_df.index = pd.to_datetime(turnout_df.index)

# Merge the DataFrames based on the 'ElectionDate' and index
merged_df = pd.merge(cleaned_df, turnout_df, left_on='ElectionDate', right_index=True, how='left')

# Now, 'merged_df' contains both the election data and the corresponding voter turnout for Miami-Dade.

merged_df

NameError: name 'cleaned_df' is not defined